we will evaluate some topic modeling techniques
- LDA
- NMF 
- Top2Vec
- Bertopic 
    - bert classic (miniml-6)
    - openai
    - tweet-classification
    - climabert
    - universal sentence encoder


for each of this i made different test with different parameters and different datasets changing the numebr of topics

Dataset:
- climate: 1669 preprocessed tweets 
- todo other

# Prepare Data

In [ ]:
from evaluation import Trainer, DataLoader
from dotenv import load_dotenv
import numpy as np
import os
import openai
from sentence_transformers import SentenceTransformer
load_dotenv()

In [2]:
dataloader = DataLoader(dataset="climate").prepare_docs(save="climate.txt").preprocess_octis(output_folder="climate")

Loading climate data
created vocab
5225
words filtering done


# NMF


In [3]:
for i, random_state in enumerate([0, 21, 42]):
    dataset = "climate"
    custom = True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="NMF",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
                      
    results = trainer.train(save=f"NMF_climate_{i+1}")

Results
npmi: -0.05603186703787161
umass: -4.021981614659724
diversity: 0.47
 
Results
npmi: -0.04093330696194557
umass: -5.894696977162936
diversity: 0.475
 
Results
npmi: -0.04475179399892415
umass: -5.726567227973454
diversity: 0.41
 
Results
npmi: -0.07069132820451146
umass: -6.639760026513971
diversity: 0.3675
 
Results
npmi: -0.05190434104957615
umass: -6.7300911945612505
diversity: 0.392
 
Results
npmi: -0.05631840316969816
umass: -3.7619125029036433
diversity: 0.5
 
Results
npmi: -0.0684209728954257
umass: -5.738021586252655
diversity: 0.43
 
Results
npmi: -0.05425098973475934
umass: -5.751620654004211
diversity: 0.37333333333333335
 
Results
npmi: -0.05317002138686715
umass: -6.506559030731209
diversity: 0.3875
 
Results
npmi: -0.07643445436005926
umass: -7.047234841772287
diversity: 0.382
 
Results
npmi: -0.0309767664156465
umass: -3.674720636876528
diversity: 0.5
 
Results
npmi: -0.07403645270712103
umass: -5.4273788221816766
diversity: 0.405
 
Results
npmi: -0.0644744531329

In [4]:
params

{'num_topics': [10, 20, 30, 40, 50], 'random_state': 42}

# LDA

In [ ]:
for i, random_state in enumerate([0, 21, 42]):
    dataset, custom = "climate", True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="LDA",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"LDA__{i+1}")

# Bertopic

 ## Data preparation 

here we prepare the data for the bertopic model, so we get back the data as a list of strings


In [15]:
%%capture
# Prepare data
dataset, custom = "climate", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

## evaluate 


Some functions to evaluate the models

In [130]:
# give a sentence_transformers model name or directly embeddings to evaluate 
def get_bertopic_result(model_name, embeddings = None, custom = False):

        # get emebddings if not provided
        display_name = model_name
        model_name = None
        results = []

        if not custom:
                model = SentenceTransformer(display_name)
                embeddings = model.encode(data, show_progress_bar=True)
                print('embedded')
                model_name = display_name

  


          # do this 3 times
        for i in range(3):
                # params that will be passed to Bertopic, model name none for custom embeddings
                params = {
                        "embedding_model": model_name,
                        "nr_topics": [(i+1)*10 for i in range(5)],  # 10, 20, 30, 40, 50 topics
                        "min_topic_size": [5,15],                   # 5, 15 documents per topic
                        "verbose": True
                }
                # train
                trainer = Trainer(      dataset=dataset,
                                        model_name="BERTopic",
                                        params=params,
                                        bt_embeddings=embeddings,
                                        custom_dataset=True,
                                        verbose=False)

                results.append(trainer.train())
                print(f"Done with {display_name} {i+1}")
        
        return results

# given the results 
def clean_results(results):
        # create result df 
        result_df = pd.DataFrame( columns = ['Dataset', 'model', 'nr_topics', 'min_topic_size', 'npmi', 'umass', 'diversity', 'computation_time'])

        # dataset used and model 
        dataset = results[0][0]['Dataset']
        model = results[0][0]['Params']['embedding_model']

        # fill the dataframe 
        for result in results:
                for test in result:
                        pd.Series([test['Dataset'], test['Params']['embedding_model'], test['Params']['nr_topics'], test['Params']['min_topic_size'], test['Scores']['npmi'], test['Scores']['umass'], test['Scores']['diversity'], test['Computation Time']], index = result_df.columns)
                        result_df = result_df.append(pd.Series([test['Dataset'], test['Params']['embedding_model'], test['Params']['nr_topics'], test['Params']['min_topic_size'], test['Scores']['npmi'], test['Scores']['umass'], test['Scores']['diversity'], test['Computation Time']], index = result_df.columns), ignore_index=True)

        # groupby and get the mean for the 3 tests 
        result_df.set_index(['Dataset', 'model', 'nr_topics', 'min_topic_size'], inplace=True)
        a = result_df.groupby(['nr_topics', 'min_topic_size']).mean()
        a.reset_index(inplace=True)

        #save the results
        a['dataset'] = dataset
        a['model'] = model
        a[['dataset', 'model','nr_topics', 'min_topic_size', 'npmi', 'umass', 'diversity',
                'computation_time']].to_csv('bertopic'+model+'.csv', index=False)
                
        return result_df

def get_openai_embeddings(texts, model="text-embedding-ada-002"):
        texts = [text.replace("\n", " ") for text in texts]

        embs = openai.Embedding.create(input = texts, model=model)['data']
        return np.array([np.array(emb['embedding']) for emb in embs])

## climatebert

In [131]:
climatebert_name = "climatebert/distilroberta-base-climate-f"
climatebert_results = get_bertopic_result(climatebert_name)
clean_results(climatebert_results)

Some weights of the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

embedded


Some weights of the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 

Done with climatebert/distilroberta-base-climate-f 1


Some weights of the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 

Done with climatebert/distilroberta-base-climate-f 2


Some weights of the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/alessiogandelli/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 

## tweetclassifcation

In [129]:
tc_name = "louisbetsch/tweetclassification-bf-model"
tc_results = get_bertopic_result(tc_name)
clean_results(tc_results)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

embedded


2023-03-16 21:18:18,285 - BERTopic - Reduced dimensionality
2023-03-16 21:18:18,835 - BERTopic - Clustered reduced embeddings
2023-03-16 21:18:50,610 - BERTopic - Reduced number of topics from 76 to 10
2023-03-16 21:19:31,827 - BERTopic - Reduced dimensionality
2023-03-16 21:19:32,025 - BERTopic - Clustered reduced embeddings
2023-03-16 21:19:33,003 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:20:11,932 - BERTopic - Reduced dimensionality
2023-03-16 21:20:12,230 - BERTopic - Clustered reduced embeddings
2023-03-16 21:20:39,171 - BERTopic - Reduced number of topics from 65 to 20
2023-03-16 21:21:18,092 - BERTopic - Reduced dimensionality
2023-03-16 21:21:18,441 - BERTopic - Clustered reduced embeddings
2023-03-16 21:21:19,654 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:21:58,640 - BERTopic - Reduced dimensionality
2023-03-16 21:21:58,919 - BERTopic - Clustered reduced embeddings
2023-03-16 21:23:10,771 - BERTopic - Reduced number of topics from 75 

Done with louisbetsch/tweetclassification-bf-model 1


2023-03-16 21:32:09,793 - BERTopic - Reduced dimensionality
2023-03-16 21:32:10,103 - BERTopic - Clustered reduced embeddings
2023-03-16 21:32:43,745 - BERTopic - Reduced number of topics from 72 to 10
2023-03-16 21:33:42,514 - BERTopic - Reduced dimensionality
2023-03-16 21:33:43,084 - BERTopic - Clustered reduced embeddings
2023-03-16 21:33:45,684 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:34:48,643 - BERTopic - Reduced dimensionality
2023-03-16 21:34:49,583 - BERTopic - Clustered reduced embeddings
2023-03-16 21:35:40,818 - BERTopic - Reduced number of topics from 69 to 20
2023-03-16 21:37:03,496 - BERTopic - Reduced dimensionality
2023-03-16 21:37:04,375 - BERTopic - Clustered reduced embeddings
2023-03-16 21:37:10,000 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:38:30,735 - BERTopic - Reduced dimensionality
2023-03-16 21:38:31,246 - BERTopic - Clustered reduced embeddings
2023-03-16 21:39:28,145 - BERTopic - Reduced number of topics from 72 

Done with louisbetsch/tweetclassification-bf-model 2


2023-03-16 21:50:19,270 - BERTopic - Reduced dimensionality
2023-03-16 21:50:20,124 - BERTopic - Clustered reduced embeddings
2023-03-16 21:51:08,993 - BERTopic - Reduced number of topics from 67 to 10
2023-03-16 21:52:37,054 - BERTopic - Reduced dimensionality
2023-03-16 21:52:37,518 - BERTopic - Clustered reduced embeddings
2023-03-16 21:52:41,048 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:54:03,326 - BERTopic - Reduced dimensionality
2023-03-16 21:54:04,107 - BERTopic - Clustered reduced embeddings
2023-03-16 21:54:59,335 - BERTopic - Reduced number of topics from 74 to 20
2023-03-16 21:56:07,246 - BERTopic - Reduced dimensionality
2023-03-16 21:56:07,510 - BERTopic - Clustered reduced embeddings
2023-03-16 21:56:09,844 - BERTopic - Reduced number of topics from 2 to 2
2023-03-16 21:57:15,014 - BERTopic - Reduced dimensionality
2023-03-16 21:57:15,649 - BERTopic - Clustered reduced embeddings
2023-03-16 21:58:41,295 - BERTopic - Reduced number of topics from 73 

Done with louisbetsch/tweetclassification-bf-model 3


npmi  \
Dataset model                                    nr_topics min_topic_size             
climate louisbetsch/tweetclassification-bf-model 10        5               0.056019   
                                                           15              0.615377   
                                                 20        5               0.015537   
                                                           15              0.615377   
                                                 30        5               0.091241   
                                                           15              0.615377   
                                                 40        5               0.037395   
                                                           15              0.615377   
                                                 50        5               0.096947   
                                                           15              0.615377   
                                                 10        5               0.014814   
                                                           15              0.615377   
                                                 20        5               0.102418   
                                                           15              0.615377   
                                                 30        5               0.111862   
                                                           15              0.615377   
                                                 40        5               0.083656   
                                                           15              0.615377   
                                                 50        5               0.073667   
                                                           15              0.615377   
                                                 10        5               0.043531   
                                                           15              0.615377   
                                                 20        5               0.032168   
                                                           15              0.615377   
                                                 30        5               0.096584   
                                                           15              0.615377   
                                                 40        5               0.104233   
                                                           15              0.615377   
                                                 50        5               0.132470   
                                                           15              0.615377   

                                                                               umass  \
Dataset model                                    nr_topics min_topic_size              
climate louisbetsch/tweetclassification-bf-model 10        5               -7.145215   
                                                           15              -0.163698   
                                                 20        5               -9.109255   
                                                           15              -0.163698   
                                                 30        5               -8.924300   
                                                           15              -0.163698   
                                                 40        5              -10.279334   
                                                           15              -0.163698   
                                                 50        5               -9.233492   
                                                           15              -0.163698   
                                                 10        5               -6.535912   
                                                           15              -0.163698   
                                                 20        5  

## bert base

In [ ]:
bert_model_name = "all-MiniLM-L6-v2"
bert_results = get_bertopic_result(bert_model_name)
clean_results(bert_results)

## openai

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai_embeddings = get_openai_embeddings(data)
openai_results = get_bertopic_result("openai", embeddings=openai_embeddings, custom=True)
clean_results(openai_results)

## Universal sentence encoder 

In [116]:
import tensorflow_hub
embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

use_embeddings = embedding_model(data)
use_embeddings = np.array([emb.numpy() for emb in use_embeddings])

2023-03-16 18:35:47.510801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 18:38:43.017828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [122]:
use_results = get_bertopic_result("use", embeddings=use_embeddings, custom=True)

2023-03-16 18:45:51,501 - BERTopic - Reduced dimensionality
2023-03-16 18:45:52,731 - BERTopic - Clustered reduced embeddings
2023-03-16 18:45:53,934 - BERTopic - Reduced number of topics from 80 to 10
2023-03-16 18:46:30,991 - BERTopic - Reduced dimensionality
2023-03-16 18:46:31,232 - BERTopic - Clustered reduced embeddings
2023-03-16 18:46:31,744 - BERTopic - Reduced number of topics from 19 to 10
2023-03-16 18:47:01,679 - BERTopic - Reduced dimensionality
2023-03-16 18:47:01,891 - BERTopic - Clustered reduced embeddings
2023-03-16 18:47:02,457 - BERTopic - Reduced number of topics from 85 to 20
2023-03-16 18:47:30,519 - BERTopic - Reduced dimensionality
2023-03-16 18:47:30,775 - BERTopic - Clustered reduced embeddings
2023-03-16 18:47:31,027 - BERTopic - Reduced number of topics from 18 to 18
2023-03-16 18:47:54,290 - BERTopic - Reduced dimensionality
2023-03-16 18:47:54,570 - BERTopic - Clustered reduced embeddings
2023-03-16 18:47:55,858 - BERTopic - Reduced number of topics from

Done with use 1


2023-03-16 18:50:46,545 - BERTopic - Reduced dimensionality
2023-03-16 18:50:46,760 - BERTopic - Clustered reduced embeddings
2023-03-16 18:50:47,268 - BERTopic - Reduced number of topics from 81 to 10
2023-03-16 18:51:03,297 - BERTopic - Reduced dimensionality
2023-03-16 18:51:03,465 - BERTopic - Clustered reduced embeddings
2023-03-16 18:51:03,651 - BERTopic - Reduced number of topics from 3 to 3
2023-03-16 18:51:23,016 - BERTopic - Reduced dimensionality
2023-03-16 18:51:23,622 - BERTopic - Clustered reduced embeddings
2023-03-16 18:51:24,640 - BERTopic - Reduced number of topics from 79 to 20
2023-03-16 18:51:49,449 - BERTopic - Reduced dimensionality
2023-03-16 18:51:49,793 - BERTopic - Clustered reduced embeddings
2023-03-16 18:51:49,999 - BERTopic - Reduced number of topics from 3 to 3
2023-03-16 18:52:13,918 - BERTopic - Reduced dimensionality
2023-03-16 18:52:14,249 - BERTopic - Clustered reduced embeddings
2023-03-16 18:52:14,825 - BERTopic - Reduced number of topics from 74 

Done with use 2


2023-03-16 18:54:42,864 - BERTopic - Reduced dimensionality
2023-03-16 18:54:43,088 - BERTopic - Clustered reduced embeddings
2023-03-16 18:54:43,542 - BERTopic - Reduced number of topics from 85 to 10
2023-03-16 18:55:01,608 - BERTopic - Reduced dimensionality
2023-03-16 18:55:01,735 - BERTopic - Clustered reduced embeddings
2023-03-16 18:55:01,866 - BERTopic - Reduced number of topics from 3 to 3
2023-03-16 18:55:15,756 - BERTopic - Reduced dimensionality
2023-03-16 18:55:15,952 - BERTopic - Clustered reduced embeddings
2023-03-16 18:55:16,356 - BERTopic - Reduced number of topics from 76 to 20
2023-03-16 18:55:32,788 - BERTopic - Reduced dimensionality
2023-03-16 18:55:32,917 - BERTopic - Clustered reduced embeddings
2023-03-16 18:55:33,037 - BERTopic - Reduced number of topics from 3 to 3
2023-03-16 18:55:48,968 - BERTopic - Reduced dimensionality
2023-03-16 18:55:49,175 - BERTopic - Clustered reduced embeddings
2023-03-16 18:55:49,703 - BERTopic - Reduced number of topics from 84 

Done with use 3


# Top2Vec

In [ ]:
for i in range(3):
    dataset, custom = "climate", True
    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              # "embedding_model": "all-MiniLM-L6-v2",
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      #custom_model=Top2Vec,
                      model_name="Top2Vec",
                      params=params,
                      verbose=False)
    results = trainer.train()

In [124]:
results

[{'Dataset': 'climate',
  'Dataset Size': 1669,
  'Model': 'Top2Vec',
  'Params': {'hdbscan_args': {'min_cluster_size': 15,
    'metric': 'euclidean',
    'cluster_selection_method': 'eom'},
   'reduction': False,
   'nr_topics': 7},
  'Scores': {'npmi': -0.09354649083222286,
   'umass': -4.349285298199307,
   'diversity': 0.44285714285714284},
  'Computation Time': 21.843358993530273},
 {'Dataset': 'climate',
  'Dataset Size': 1669,
  'Model': 'Top2Vec',
  'Params': {'hdbscan_args': {'min_cluster_size': 15,
    'metric': 'euclidean',
    'cluster_selection_method': 'eom'},
   'reduction': False,
   'nr_topics': 9},
  'Scores': {'npmi': -0.15375534843485045,
   'umass': -6.19505904468258,
   'diversity': 0.36666666666666664},
  'Computation Time': 30.2948739528656},
 {'Dataset': 'climate',
  'Dataset Size': 1669,
  'Model': 'Top2Vec',
  'Params': {'hdbscan_args': {'min_cluster_size': 15,
    'metric': 'euclidean',
    'cluster_selection_method': 'eom'},
   'reduction': False,
   'nr_to

In [126]:
# create result df 
result_df = pd.DataFrame( columns = ['Dataset', 'model', 'nr_topics', 'min_topic_size', 'npmi', 'umass', 'diversity', 'computation_time'])

# dataset used and model 
dataset = results[0]['Dataset']
model = results[0]['Model']

# fill the dataframe 
for result in results:
        for test in result:
                row = pd.Series([test['Dataset'], test['Model'], test['Params']['nr_topics'], test['Params']['hdbscan_args']['min_claster_size'], test['Scores']['npmi'], test['Scores']['umass'], test['Scores']['diversity'], test['Computation Time']], index = result_df.columns)
                result_df = result_df.append(row, ignore_index=True)

# groupby and get the mean for the 3 tests 
result_df.set_index(['Dataset', 'model', 'nr_topics', 'min_topic_size'], inplace=True)
a = result_df.groupby(['nr_topics', 'min_topic_size']).mean()
a.reset_index(inplace=True)

#save the results
a['dataset'] = dataset
a['model'] = model
a[['dataset', 'model','nr_topics', 'min_topic_size', 'npmi', 'umass', 'diversity',
        'computation_time']].to_csv('bertopic'+'openai'+'.csv', index=False)
        
return result_df

TypeError: string indices must be integers